# SBTi-Finance Tool - What-if Analysis
In this notebook we show you how to analyse what impact different actions would have on the temperature score of your portfolio.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details on the different what-if scenarios.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 


## Setting up
First we will set up the imports, data providers, and load the portfolio. 

For more examples of this process, please refer to notebook 1 & 2 ([analysis](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) and [quick calculation example](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/2_quick_temp_score_calculation.ipynb)).

In [ ]:
%pip install sbti-finance-tool

In [ ]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd
import requests
from datetime import datetime

In [ ]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

# Print original columns to diagnose
print("Original columns:", df_portfolio.columns.tolist())

# Create a more cautious renaming dictionary based on what columns exist
rename_dict = {}
if "Company Name" in df_portfolio.columns:
    rename_dict["Company Name"] = "company_name"
if "ISIN" in df_portfolio.columns:
    rename_dict["ISIN"] = "isin" 
if "LEI" in df_portfolio.columns:
    rename_dict["LEI"] = "lei"
if "Sector" in df_portfolio.columns:
    rename_dict["Sector"] = "sector"
if "Target" in df_portfolio.columns:
    rename_dict["Target"] = "full_target_language"
if "Net-Zero Committed" in df_portfolio.columns:
    rename_dict["Net-Zero Committed"] = "net_zero_status"
if "Near term - Target Status" in df_portfolio.columns:
    rename_dict["Near term - Target Status"] = "near_term_status"
if "Target Classification" in df_portfolio.columns:
    rename_dict["Target Classification"] = "target_classification_long"
if "Extension" in df_portfolio.columns:
    rename_dict["Extension"] = "reason_for_extension_or_removal"
if "Date" in df_portfolio.columns:
    rename_dict["Date"] = "date_updated"

# Apply column renaming
df_portfolio = df_portfolio.rename(columns=rename_dict)

# Handle legacy column names if they exist
if 'company_isin' in df_portfolio.columns:
    df_portfolio.rename(columns={'company_isin': 'isin'}, inplace=True)
if 'company_lei' in df_portfolio.columns:
    df_portfolio.rename(columns={'company_lei': 'lei'}, inplace=True)

# Ensure required columns exist
required_columns = ['company_id', 'company_name', 'isin', 'lei', 'investment_value']
for col in required_columns:
    if col not in df_portfolio.columns:
        df_portfolio[col] = None

# Convert identifiers to string
if 'isin' in df_portfolio.columns:
    df_portfolio['isin'] = df_portfolio['isin'].astype(str)
if 'lei' in df_portfolio.columns:
    df_portfolio['lei'] = df_portfolio['lei'].astype(str)

# Check for duplicate values in the 'company_id' column
duplicate_ids = df_portfolio[df_portfolio.duplicated('company_id', keep=False)]
if not duplicate_ids.empty:
    print("Error: Duplicate values found in the 'company_id' column:")
    print(duplicate_ids)
else:
    print("No duplicate values found in the 'company_id' column.")

# Display final columns to verify
print("Final columns:", df_portfolio.columns.tolist())

# Convert to portfolio objects and get data
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
try:
    portfolio_data = SBTi.utils.get_data([provider], companies)
    scenarios = {}
    print("Successfully loaded portfolio data")
except Exception as e:
    print(f"Error loading portfolio data: {str(e)}")
    # Add more error handling if needed
    import traceback
    traceback.print_exc()

Import the CTA and set up a SBTi target frame of reference.

In [ ]:
#Provides an absolute frame of reference for SBTi targets so that they are considered as cardinal compared to others in the calculation of temperature scores.
def inject_sbti_validation_for_timeframe_scope_data(amended_portfolio, original_portfolio, debug=True):
    """
    Specially designed for the SBTi tool where amended_portfolio contains multiple rows 
    per company (one for each time frame and scope combination).
    """
    if 'sbti_validated' not in original_portfolio.columns:
        print("⚠ No 'sbti_validated' column found in original portfolio")
        return amended_portfolio
    
    # Store original values before modification
    original_validated_count = original_portfolio['sbti_validated'].sum()
    original_companies_count = len(original_portfolio)
    
    # Get count of unique companies in amended portfolio
    unique_companies_amended = amended_portfolio['company_id'].nunique()
    
    if debug:
        print(f"Original portfolio: {original_companies_count} companies, {original_validated_count} validated")
        print(f"Amended portfolio: {len(amended_portfolio)} rows, {unique_companies_amended} unique companies")
        
        # Check for duplicated rows by company_id
        if len(amended_portfolio) > unique_companies_amended:
            print(f"Multiple rows per company detected in amended portfolio")
            print(f"Rows per company: {len(amended_portfolio) / unique_companies_amended:.2f}")
            
            # Show distribution of time_frame and scope if they exist
            if 'time_frame' in amended_portfolio.columns:
                print("\nTime frame distribution:")
                print(amended_portfolio['time_frame'].value_counts())
            if 'scope' in amended_portfolio.columns:
                print("\nScope distribution:")
                print(amended_portfolio['scope'].value_counts())
    
    # Create a validation mapping
    validation_map = dict(zip(original_portfolio['company_id'], original_portfolio['sbti_validated']))
    
    # Apply validation to all rows in amended portfolio
    amended_portfolio['sbti_validated'] = amended_portfolio['company_id'].map(validation_map).fillna(False)
    
    # Count unique validated companies after modification
    validated_companies = amended_portfolio[amended_portfolio['sbti_validated']]['company_id'].nunique()
    
    # Print validation summary
    print(f"\nOriginal validated companies: {original_validated_count}")
    print(f"Unique companies validated in amended portfolio: {validated_companies}")
    
    if original_validated_count == validated_companies:
        print("✓ CTA validation successfully preserved at company level")
    else:
        print("⚠ CTA validation mismatch at company level")
        
        # Additional debugging
        if debug:
            print("\nChecking for specific discrepancies...")
            # Get list of company IDs that should be validated
            original_validated_ids = set(original_portfolio[original_portfolio['sbti_validated']]['company_id'])
            amended_validated_ids = set(amended_portfolio[amended_portfolio['sbti_validated']]['company_id'])
            
            missing_validations = original_validated_ids - amended_validated_ids
            extra_validations = amended_validated_ids - original_validated_ids
            
            if missing_validations:
                print(f"Companies that should be validated but aren't: {len(missing_validations)}")
                print(missing_validations)
            
            if extra_validations:
                print(f"Companies that shouldn't be validated but are: {len(extra_validations)}")
                print(extra_validations)
    
    return amended_portfolio
# STANDALONE SBTi VALIDATION - Download and process CTA data
print("Downloading SBTi Companies Taking Action (CTA) data...")
CTA_FILE_URL = "https://cdn.sciencebasedtargets.org/download/target-dashboard"

try:
    resp = requests.get(CTA_FILE_URL)
    if resp.status_code == 200:
        cta_file = pd.read_excel(resp.content)
        print(f"Downloaded CTA data with {len(cta_file)} rows")
        
        # Extract relevant columns
        targets = cta_file[['company_name', 'isin', 'lei', 'action', 'target', 'date_published']]
        
        # Filter for companies with targets
        companies_with_targets = targets[targets['action'] == 'Target']
        
        # Get unique identifiers
        all_isin_set = set(companies_with_targets['isin'].dropna())
        all_lei_set = set(companies_with_targets['lei'].dropna())
        
        # Create a set of lowercase company names
        companies_with_targets['company_name_lower'] = companies_with_targets['company_name'].str.lower()
        company_name_set = set(companies_with_targets['company_name_lower'].dropna())
        
        # Add portfolio columns if they don't exist
        if 'isin' not in df_portfolio.columns and 'company_isin' in df_portfolio.columns:
            df_portfolio['isin'] = df_portfolio['company_isin']
        if 'lei' not in df_portfolio.columns and 'company_lei' in df_portfolio.columns:
            df_portfolio['lei'] = df_portfolio['company_lei']
        
        # Function to check if ISIN, LEI, or company name is validated
        def is_validated(row):
            # First check LEI
            if pd.notna(row.get('lei')) and str(row.get('lei')).lower() in [str(x).lower() for x in all_lei_set]:
                return True
            
            # Then check ISIN
            if pd.notna(row.get('isin')) and str(row.get('isin')).lower() in [str(x).lower() for x in all_isin_set]:
                return True
            
            # Finally check company name
            if pd.notna(row.get('company_name')):
                company_name_lower = str(row.get('company_name')).lower()
                if company_name_lower in company_name_set:
                    return True
            
            return False
        
        # Add the validated column to the portfolio
        df_portfolio['sbti_validated'] = df_portfolio.apply(is_validated, axis=1)
        
        # Convert portfolio to company objects again (after adding sbti_validated)
        companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
        
        # Print validation summary
        validated_count = df_portfolio['sbti_validated'].sum()
        print(f"Companies with SBTi-validated targets: {validated_count} out of {len(df_portfolio)} ({validated_count/len(df_portfolio)*100:.2f}%)")
        
        if 'investment_value' in df_portfolio.columns:
            total_investment = df_portfolio['investment_value'].sum()
            validated_investment = df_portfolio[df_portfolio['sbti_validated']]['investment_value'].sum()
            print(f"Portfolio coverage by investment value: {validated_investment/total_investment*100:.2f}%")
    else:
        print(f"Failed to download CTA file: HTTP {resp.status_code}")
except Exception as e:
    print(f"Error processing CTA file: {str(e)}")

# Update provider data with our validation results
print("Updating provider data with validated companies...")
try:
    # Get the fundamental_data from provider
    if hasattr(provider, 'data') and 'fundamental_data' in provider.data:
        # Create a mapping of company_id to validation status
        validation_map = df_portfolio[['company_id', 'sbti_validated']].set_index('company_id')['sbti_validated'].to_dict()
        
        # Update sbti_validated in provider data
        updated_count = 0
        for idx, row in provider.data['fundamental_data'].iterrows():
            company_id = row['company_id']
            if company_id in validation_map:
                # Ensure we're setting a proper boolean value
                is_validated = bool(validation_map[company_id])
                provider.data['fundamental_data'].at[idx, 'sbti_validated'] = is_validated
                updated_count += 1
        
        # Force the sbti_validated column to be boolean type
        provider.data['fundamental_data']['sbti_validated'] = provider.data['fundamental_data']['sbti_validated'].astype(bool)
        
        print(f"Updated sbti_validated for {updated_count} companies in provider data")
    else:
        print("Provider does not have expected data structure - sbti_validated flags may be overwritten")
except Exception as e:
    print(f"Error updating provider data: {str(e)}")

## Base scenario
To determine what the effect is of running a certain scenario, we first need a base line. To do so, first we calculate the portfolio temperature score, without defining a scenario. 

We can then analyse the effect of the actions in our scenario's by comparing the temperature score medium-term timeframe in the combined s1s2s3 scope. Of course you can use multiple, or different criteria and also change the scope used to calculate the scores.

In [ ]:
# Set the time frames and scopes for the scoring          other options displayed below. in this NB we use MID and S1S2S3
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # ETimeFrames: SHORT MID and LONG
scopes = [EScope.S1S2S3]                                # EScopes: S1, S2, S3, S1S2 and S1S2S3

Below we compute the temperature scores for all the timeframe and scope combinations. Subsequently, one can query the score of interest as shown in the last line in the cell below.

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
base_scenario = inject_sbti_validation_for_timeframe_scope_data(base_scenario, df_portfolio, debug=True)
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_score = base_scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Base Scenario': base_score})
print("The portfolio's base scenario temperature score (mid - S1S2S3) is {c:.2f} degrees celcius".format(c=base_score))

## Scenario 1
In scenario 1 we engage companies to set emissions reduction targets of at least 2 degrees Celsius. 

In practice, in the model, this means that we change the default score (i.e. the score when a company didn't set a target) to 2 degrees Celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_one = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
scenario_one = inject_sbti_validation_for_timeframe_scope_data(scenario_one, df_portfolio, debug=True)
scenario_aggregated = temperature_score.aggregate_scores(scenario_one)
scenario_1_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 1': scenario_1_score})
print("The portfolio's scenario 1 temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_1_score))

## Scenario 2
In scenario 2 we engage with companies to influence them to set SBTi approved emissions reduction targets. This means that the maximum score for all targets will be capped at 1.75 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_two = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
scenario_two = inject_sbti_validation_for_timeframe_scope_data(scenario_two, df_portfolio, debug=True)
scenario_aggregated = temperature_score.aggregate_scores(scenario_two)
scenario_2_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 2': scenario_2_score})
print("The portfolio's scenario 2 temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_2_score))

## Scenario 3a
In scenario 3a we engage the top 10 contributing companies to the portfolio temperature score to influence them to set emission reduction targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_three_a = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
scenario_three_a = inject_sbti_validation_for_timeframe_scope_data(scenario_three_a, df_portfolio, debug=True)
scenario_aggregated = temperature_score.aggregate_scores(scenario_three_a)
scenario_3a_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 3a': scenario_3a_score})
print("The portfolio's scenario 3a temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_3a_score))

## Scenario 3b
In scenario 3b we engage the top 10 contributing companies to the portfolio temperature score to influence them to set SBTi-approved targets. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_three_b = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
scenario_three_b = inject_sbti_validation_for_timeframe_scope_data(scenario_three_b, df_portfolio, debug=True)
scenario_aggregated = temperature_score.aggregate_scores(scenario_three_b)
scenario_3b_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 3b': scenario_3b_score})
print("The portfolio's scenario 3b temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_3b_score))

## Scenario 4a
In scenario 4a we engage with any company in the portfolio to influence them to set emission reduction targets. We define these companies by adding "`TRUE`" in the `engagement_target`-column in the portfolio input-file. 

This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees Celsius.

Scenario 4a and 4b gives you a lot of flexibility, as you can define the companies with which you engage freely and for example focus your engagement efforts on certain sectors, countries, market cap buckets to ensure that you spend your time wisely. 

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_four_a = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
scenario_four_a = inject_sbti_validation_for_timeframe_scope_data(scenario_four_a, df_portfolio, debug=True)
scenario_aggregated = temperature_score.aggregate_scores(scenario_four_a)
scenario_4a_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 4a': scenario_4a_score})
print("The portfolio's scenario 4a temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_4a_score))

## Scenario 4b
In scenario 4b we can engage with any company in the portfolio and manage to influence them to set SBTi-approved targets. We define these companies by adding "`TRUE`" in the `engagement_target`-column in the portfolio input-file. 

This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees Celsius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_four_b = temperature_score.calculate(portfolio_data.copy())
# Preserve the CTA validation from our direct download
scenario_four_b = inject_sbti_validation_for_timeframe_scope_data(scenario_four_b, df_portfolio, debug=True)
scenario_aggregated = temperature_score.aggregate_scores(scenario_four_b)
scenario_4b_score = scenario_aggregated.mid.S1S2S3.all.score
scenarios.update({'Scenario 4b': scenario_4b_score})
print("The portfolio's scenario 4b temperature score (mid - S1S2S3) is {c:.2f} degrees Celsius".format(c=scenario_4b_score))

To see which scenario provides the most suitable outcome we print an overview of all scenarios:

In [ ]:
print('{:<20s}{}'.format('', 'Temp score'))
print('{:<20s}{}'.format('Scenario overview', '(mid - S1S2S3):'))
for scenario, score in scenarios.items():
    print('{:<20s}{:.2f}'.format(scenario, score))